In [1]:
import flwr as fl
import utils
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from typing import Dict

"""
The Flower server interacts with clients through an interface called Client. 
When the server selects a particular client for training, 
it sends training instructions over the network. 
The client receives those instructions and calls one of the Client methods to run your code 
"""
arr="se "

#Send round number to client.
def fit_round(rnd: int) -> Dict:
    return {"rnd": rnd}
  

def get_eval_fn(model: LogisticRegression):
    """Return an evaluation function for server-side evaluation."""

    # Load test data here to avoid the overhead of doing it in `evaluate` itself
    _, (X_test, y_test) = utils.load_mnist()

    # The `evaluate` function will be called after every round
    def evaluate(parameters: fl.common.Weights):
        # Update model with the latest parameters
        utils.set_model_params(model, parameters)
        loss = log_loss(y_test, model.predict_proba(X_test))
        accuracy = model.score(X_test, y_test)
        global arr
        # 存住 正確率
        ac = model.score(X_test, y_test)
        arr = arr + str(ac) + ' '
        return loss, {"accuracy": accuracy}

    return evaluate


# Start Flower server for five rounds of federated learning
if __name__ == "__main__":
    model = LogisticRegression()
    utils.set_initial_params(model)

    #strategy(演算法戰略)這邊使用FedAvg
    strategy = fl.server.strategy.FedAvg(
        min_available_clients=2,
        eval_fn=get_eval_fn(model),
        on_fit_config_fn=fit_round,
    )
    fl.server.start_server("0.0.0.0:8080", strategy=strategy, config={"num_rounds": 5},force_final_distributed_eval="true")#force_final_distributed_eval

    f = open("output.txt", "a")
    f.writelines(arr)
    f.writelines("\n")

    # 關閉檔案 
    f.close()

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[7. 2. 1. ... 4. 5. 6.]


INFO flower 2022-04-14 10:58:27,287 | app.py:109 | Flower server running (5 rounds)
SSL is disabled
INFO flower 2022-04-14 10:58:27,288 | server.py:128 | Initializing global parameters
INFO flower 2022-04-14 10:58:27,289 | server.py:327 | Requesting initial parameters from one random client
INFO flower 2022-04-14 10:59:21,982 | server.py:330 | Received initial parameters from one random client
INFO flower 2022-04-14 10:59:21,985 | server.py:130 | Evaluating initial parameters
INFO flower 2022-04-14 10:59:22,865 | server.py:133 | initial parameters (loss, other metrics): 2.3025850929940455, {'accuracy': 0.098}
INFO flower 2022-04-14 10:59:22,867 | server.py:143 | FL starting
DEBUG flower 2022-04-14 10:59:32,306 | server.py:265 | fit_round: strategy sampled 2 clients (out of 2)
DEBUG flower 2022-04-14 10:59:39,134 | server.py:277 | fit_round received 2 results and 0 failures
INFO flower 2022-04-14 10:59:39,822 | server.py:158 | fit progress: (1, 1.3243348780603164, {'accuracy': 0.6841}, 